# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """    
    eos = source_vocab_to_int['<EOS>']
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    target_text = [[target_vocab_to_int[word] for word in sentence.split()] + [eos] for sentence in target_text.split('\n')]

    return source_id_text, target_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    target_sequence_length = tf.placeholder(tf.int32, (None,), name="target_sequence_length")
    max_target_len = tf.reduce_max(target_sequence_length, name="max_target_len")
    source_sequence_length = tf.placeholder(tf.int32, (None,), name="source_sequence_length")

    return input_data, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    go_id = target_vocab_to_int['<GO>']
    last_slice = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    dec_input = tf.concat([tf.fill(
        [batch_size, 1], go_id), last_slice], 1)
    
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


# Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    def lstm_cell(rnn_size, keep_prob=keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    embedded_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    rnn_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    output, state = tf.nn.dynamic_rnn(rnn_cells, embedded_input,
                                                     sequence_length=source_sequence_length,
                                                     dtype=tf.float32)
    return output, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length, time_major=False)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer=output_layer)
    final_outputs = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)[0]
    return final_outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    final_outputs = tf.contrib.seq2seq.dynamic_decode(decoder,
                                             impute_finished = True,
                                             maximum_iterations = max_target_sequence_length)[0]
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [15]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def lstm_cell(rnn_size, keep_prob=keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    embedded_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cell_stack = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope('decode'):
        basic_decoder_output_train = decoding_layer_train(encoder_state, cell_stack, embedded_input, 
                         target_sequence_length, max_target_sequence_length, 
                         output_layer, keep_prob)
        
    with tf.variable_scope('decode', reuse=True):
        basic_decoder_output_infer = decoding_layer_infer(encoder_state, cell_stack, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)    
    
    return basic_decoder_output_train, basic_decoder_output_infer



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [16]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, enc_embedding_size)
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    basic_decoder_out_train, basic_decoder_out_inf = decoding_layer(dec_input, state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    return basic_decoder_out_train, basic_decoder_out_inf


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [5]:
# Number of Epochs
epochs = 50
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/269 - Train Accuracy: 0.2327, Validation Accuracy: 0.3096, Loss: 5.5793
Epoch   0 Batch   20/269 - Train Accuracy: 0.2392, Validation Accuracy: 0.3113, Loss: 4.7674
Epoch   0 Batch   30/269 - Train Accuracy: 0.2768, Validation Accuracy: 0.3220, Loss: 4.0470
Epoch   0 Batch   40/269 - Train Accuracy: 0.2712, Validation Accuracy: 0.3399, Loss: 3.7689
Epoch   0 Batch   50/269 - Train Accuracy: 0.2817, Validation Accuracy: 0.3411, Loss: 3.6042
Epoch   0 Batch   60/269 - Train Accuracy: 0.3260, Validation Accuracy: 0.3420, Loss: 3.3306
Epoch   0 Batch   70/269 - Train Accuracy: 0.3194, Validation Accuracy: 0.3428, Loss: 3.3101
Epoch   0 Batch   80/269 - Train Accuracy: 0.3192, Validation Accuracy: 0.3427, Loss: 3.2433
Epoch   0 Batch   90/269 - Train Accuracy: 0.2665, Validation Accuracy: 0.3426, Loss: 3.3753
Epoch   0 Batch  100/269 - Train Accuracy: 0.3219, Validation Accuracy: 0.3426, Loss: 3.0980
Epoch   0 Batch  110/269 - Train Accuracy: 0.3145, Validation Accuracy

Epoch   3 Batch  120/269 - Train Accuracy: 0.4922, Validation Accuracy: 0.5268, Loss: 1.6560
Epoch   3 Batch  130/269 - Train Accuracy: 0.4760, Validation Accuracy: 0.5294, Loss: 1.6693
Epoch   3 Batch  140/269 - Train Accuracy: 0.5166, Validation Accuracy: 0.5286, Loss: 1.5605
Epoch   3 Batch  150/269 - Train Accuracy: 0.5122, Validation Accuracy: 0.5342, Loss: 1.5489
Epoch   3 Batch  160/269 - Train Accuracy: 0.5155, Validation Accuracy: 0.5392, Loss: 1.5412
Epoch   3 Batch  170/269 - Train Accuracy: 0.5166, Validation Accuracy: 0.5387, Loss: 1.5154
Epoch   3 Batch  180/269 - Train Accuracy: 0.5187, Validation Accuracy: 0.5403, Loss: 1.4931
Epoch   3 Batch  190/269 - Train Accuracy: 0.5176, Validation Accuracy: 0.5358, Loss: 1.4807
Epoch   3 Batch  200/269 - Train Accuracy: 0.5075, Validation Accuracy: 0.5413, Loss: 1.5339
Epoch   3 Batch  210/269 - Train Accuracy: 0.5169, Validation Accuracy: 0.5328, Loss: 1.4539
Epoch   3 Batch  220/269 - Train Accuracy: 0.5276, Validation Accuracy

Epoch   6 Batch  230/269 - Train Accuracy: 0.5608, Validation Accuracy: 0.5607, Loss: 0.8714
Epoch   6 Batch  240/269 - Train Accuracy: 0.5892, Validation Accuracy: 0.5647, Loss: 0.8100
Epoch   6 Batch  250/269 - Train Accuracy: 0.5554, Validation Accuracy: 0.5775, Loss: 0.8865
Epoch   6 Batch  260/269 - Train Accuracy: 0.5525, Validation Accuracy: 0.5695, Loss: 0.9012
Epoch   7 Batch   10/269 - Train Accuracy: 0.5577, Validation Accuracy: 0.5730, Loss: 0.8732
Epoch   7 Batch   20/269 - Train Accuracy: 0.5576, Validation Accuracy: 0.5738, Loss: 0.8839
Epoch   7 Batch   30/269 - Train Accuracy: 0.5703, Validation Accuracy: 0.5759, Loss: 0.8332
Epoch   7 Batch   40/269 - Train Accuracy: 0.5624, Validation Accuracy: 0.5741, Loss: 0.8695
Epoch   7 Batch   50/269 - Train Accuracy: 0.5485, Validation Accuracy: 0.5758, Loss: 0.8768
Epoch   7 Batch   60/269 - Train Accuracy: 0.5821, Validation Accuracy: 0.5756, Loss: 0.8041
Epoch   7 Batch   70/269 - Train Accuracy: 0.5724, Validation Accuracy

Epoch  10 Batch   80/269 - Train Accuracy: 0.6267, Validation Accuracy: 0.6277, Loss: 0.6608
Epoch  10 Batch   90/269 - Train Accuracy: 0.5908, Validation Accuracy: 0.6387, Loss: 0.7049
Epoch  10 Batch  100/269 - Train Accuracy: 0.6548, Validation Accuracy: 0.6313, Loss: 0.6480
Epoch  10 Batch  110/269 - Train Accuracy: 0.6149, Validation Accuracy: 0.6369, Loss: 0.6510
Epoch  10 Batch  120/269 - Train Accuracy: 0.6152, Validation Accuracy: 0.6416, Loss: 0.6828
Epoch  10 Batch  130/269 - Train Accuracy: 0.5981, Validation Accuracy: 0.6309, Loss: 0.6914
Epoch  10 Batch  140/269 - Train Accuracy: 0.6305, Validation Accuracy: 0.6299, Loss: 0.6645
Epoch  10 Batch  150/269 - Train Accuracy: 0.6378, Validation Accuracy: 0.6257, Loss: 0.6605
Epoch  10 Batch  160/269 - Train Accuracy: 0.6174, Validation Accuracy: 0.6316, Loss: 0.6545
Epoch  10 Batch  170/269 - Train Accuracy: 0.6292, Validation Accuracy: 0.6419, Loss: 0.6525
Epoch  10 Batch  180/269 - Train Accuracy: 0.6299, Validation Accuracy

Epoch  13 Batch  190/269 - Train Accuracy: 0.6596, Validation Accuracy: 0.6588, Loss: 0.5159
Epoch  13 Batch  200/269 - Train Accuracy: 0.6575, Validation Accuracy: 0.6670, Loss: 0.5407
Epoch  13 Batch  210/269 - Train Accuracy: 0.6772, Validation Accuracy: 0.6641, Loss: 0.5207
Epoch  13 Batch  220/269 - Train Accuracy: 0.6680, Validation Accuracy: 0.6698, Loss: 0.5000
Epoch  13 Batch  230/269 - Train Accuracy: 0.6652, Validation Accuracy: 0.6674, Loss: 0.5165
Epoch  13 Batch  240/269 - Train Accuracy: 0.6857, Validation Accuracy: 0.6663, Loss: 0.4810
Epoch  13 Batch  250/269 - Train Accuracy: 0.6646, Validation Accuracy: 0.6635, Loss: 0.5321
Epoch  13 Batch  260/269 - Train Accuracy: 0.6521, Validation Accuracy: 0.6705, Loss: 0.5516
Epoch  14 Batch   10/269 - Train Accuracy: 0.6664, Validation Accuracy: 0.6740, Loss: 0.5296
Epoch  14 Batch   20/269 - Train Accuracy: 0.6684, Validation Accuracy: 0.6740, Loss: 0.5369
Epoch  14 Batch   30/269 - Train Accuracy: 0.6769, Validation Accuracy

...

Epoch  34 Batch   70/269 - Train Accuracy: 0.8742, Validation Accuracy: 0.8832, Loss: 0.1908
Epoch  34 Batch   80/269 - Train Accuracy: 0.8922, Validation Accuracy: 0.8856, Loss: 0.1950
Epoch  34 Batch   90/269 - Train Accuracy: 0.8570, Validation Accuracy: 0.8802, Loss: 0.2077
Epoch  34 Batch  100/269 - Train Accuracy: 0.8906, Validation Accuracy: 0.8817, Loss: 0.1886
Epoch  34 Batch  110/269 - Train Accuracy: 0.8750, Validation Accuracy: 0.8888, Loss: 0.1884
Epoch  34 Batch  120/269 - Train Accuracy: 0.8762, Validation Accuracy: 0.8839, Loss: 0.1970
Epoch  34 Batch  130/269 - Train Accuracy: 0.8666, Validation Accuracy: 0.8827, Loss: 0.1985
Epoch  34 Batch  140/269 - Train Accuracy: 0.8684, Validation Accuracy: 0.8845, Loss: 0.2084
Epoch  34 Batch  150/269 - Train Accuracy: 0.8729, Validation Accuracy: 0.8813, Loss: 0.1885
Epoch  34 Batch  160/269 - Train Accuracy: 0.8783, Validation Accuracy: 0.8918, Loss: 0.1998
Epoch  34 Batch  170/269 - Train Accuracy: 0.8702, Validation Accuracy

Epoch  37 Batch  180/269 - Train Accuracy: 0.8880, Validation Accuracy: 0.8918, Loss: 0.1679
Epoch  37 Batch  190/269 - Train Accuracy: 0.8831, Validation Accuracy: 0.8917, Loss: 0.1704
Epoch  37 Batch  200/269 - Train Accuracy: 0.8797, Validation Accuracy: 0.8936, Loss: 0.1749
Epoch  37 Batch  210/269 - Train Accuracy: 0.8910, Validation Accuracy: 0.8923, Loss: 0.1638
Epoch  37 Batch  220/269 - Train Accuracy: 0.8918, Validation Accuracy: 0.8928, Loss: 0.1672
Epoch  37 Batch  230/269 - Train Accuracy: 0.8849, Validation Accuracy: 0.8950, Loss: 0.1640
Epoch  37 Batch  240/269 - Train Accuracy: 0.8944, Validation Accuracy: 0.8916, Loss: 0.1588
Epoch  37 Batch  250/269 - Train Accuracy: 0.9006, Validation Accuracy: 0.8940, Loss: 0.1649
Epoch  37 Batch  260/269 - Train Accuracy: 0.8749, Validation Accuracy: 0.8968, Loss: 0.1884
Epoch  38 Batch   10/269 - Train Accuracy: 0.8809, Validation Accuracy: 0.8945, Loss: 0.1650
Epoch  38 Batch   20/269 - Train Accuracy: 0.8822, Validation Accuracy

Epoch  41 Batch   30/269 - Train Accuracy: 0.9026, Validation Accuracy: 0.9016, Loss: 0.1516
Epoch  41 Batch   40/269 - Train Accuracy: 0.8858, Validation Accuracy: 0.8969, Loss: 0.1633
Epoch  41 Batch   50/269 - Train Accuracy: 0.8743, Validation Accuracy: 0.9012, Loss: 0.1711
Epoch  41 Batch   60/269 - Train Accuracy: 0.8923, Validation Accuracy: 0.9008, Loss: 0.1403
Epoch  41 Batch   70/269 - Train Accuracy: 0.8929, Validation Accuracy: 0.8947, Loss: 0.1528
Epoch  41 Batch   80/269 - Train Accuracy: 0.9082, Validation Accuracy: 0.9026, Loss: 0.1501
Epoch  41 Batch   90/269 - Train Accuracy: 0.8771, Validation Accuracy: 0.9028, Loss: 0.1620
Epoch  41 Batch  100/269 - Train Accuracy: 0.9095, Validation Accuracy: 0.8908, Loss: 0.1494
Epoch  41 Batch  110/269 - Train Accuracy: 0.8973, Validation Accuracy: 0.9055, Loss: 0.1458
Epoch  41 Batch  120/269 - Train Accuracy: 0.8920, Validation Accuracy: 0.9012, Loss: 0.1539
Epoch  41 Batch  130/269 - Train Accuracy: 0.8888, Validation Accuracy

Epoch  44 Batch  140/269 - Train Accuracy: 0.8947, Validation Accuracy: 0.8975, Loss: 0.1537
Epoch  44 Batch  150/269 - Train Accuracy: 0.9021, Validation Accuracy: 0.9082, Loss: 0.1420
Epoch  44 Batch  160/269 - Train Accuracy: 0.9011, Validation Accuracy: 0.9055, Loss: 0.1386
Epoch  44 Batch  170/269 - Train Accuracy: 0.8887, Validation Accuracy: 0.9000, Loss: 0.1382
Epoch  44 Batch  180/269 - Train Accuracy: 0.9112, Validation Accuracy: 0.8947, Loss: 0.1347
Epoch  44 Batch  190/269 - Train Accuracy: 0.9022, Validation Accuracy: 0.9016, Loss: 0.1345
Epoch  44 Batch  200/269 - Train Accuracy: 0.8945, Validation Accuracy: 0.9017, Loss: 0.1399
Epoch  44 Batch  210/269 - Train Accuracy: 0.9039, Validation Accuracy: 0.9063, Loss: 0.1380
Epoch  44 Batch  220/269 - Train Accuracy: 0.9081, Validation Accuracy: 0.8980, Loss: 0.1367
Epoch  44 Batch  230/269 - Train Accuracy: 0.8985, Validation Accuracy: 0.9017, Loss: 0.1331
Epoch  44 Batch  240/269 - Train Accuracy: 0.9115, Validation Accuracy

Epoch  47 Batch  250/269 - Train Accuracy: 0.9196, Validation Accuracy: 0.9096, Loss: 0.1236
Epoch  47 Batch  260/269 - Train Accuracy: 0.9019, Validation Accuracy: 0.9141, Loss: 0.1413
Epoch  48 Batch   10/269 - Train Accuracy: 0.9039, Validation Accuracy: 0.9058, Loss: 0.1204
Epoch  48 Batch   20/269 - Train Accuracy: 0.9058, Validation Accuracy: 0.9070, Loss: 0.1282
Epoch  48 Batch   30/269 - Train Accuracy: 0.9130, Validation Accuracy: 0.9023, Loss: 0.1206
Epoch  48 Batch   40/269 - Train Accuracy: 0.8939, Validation Accuracy: 0.9021, Loss: 0.1358
Epoch  48 Batch   50/269 - Train Accuracy: 0.8607, Validation Accuracy: 0.8684, Loss: 0.1474
Epoch  48 Batch   60/269 - Train Accuracy: 0.8983, Validation Accuracy: 0.9059, Loss: 0.1489
Epoch  48 Batch   70/269 - Train Accuracy: 0.9084, Validation Accuracy: 0.9015, Loss: 0.1398
Epoch  48 Batch   80/269 - Train Accuracy: 0.9112, Validation Accuracy: 0.9055, Loss: 0.1277
Epoch  48 Batch   90/269 - Train Accuracy: 0.8896, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL.

"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()


## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [3]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    unk_int = vocab_to_int['<UNK>']
    return [vocab_to_int.get(word, unk_int) for word in sentence.lower().split(' ')]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [8]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [46, 130, 12, 13, 144, 94, 193]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [100, 100, 100, 100, 100, 100, 314, 295, 27, 149, 169, 1]
  French Words: a a a a a a grosse voiture cours notre . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.